# DEEPFAKE DETECTION PROJECT

In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

In [0]:
!pip3 install poutyne
!pip3 install efficientnet-pytorch

In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/deepfake/')

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from PIL import Image
from tqdm import tqdm

from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
from functools import partial

from blazeface import BlazeFace

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from efficientnet_pytorch import EfficientNet

from poutyne.framework import Model, ModelCheckpoint, BestModelRestore, CSVLogger

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Datasets Preprocessing

In [0]:
FFPP_SRC='/content/gdrive/My Drive/Colab Notebooks/deepfake/datasets'
VIDEODF_SRC = '/content/gdrive/My Drive/Colab Notebooks/deepfake/datasets/ffpp_videos.pkl'

BLAZEFACE_WEIGHTS = "/content/gdrive/My Drive/Colab Notebooks/deepfake/blazeface/blazeface.pth"
BLAZEFACE_ANCHORS = "/content/gdrive/My Drive/Colab Notebooks/deepfake/blazeface/anchors.npy"

FACES_DST = '/content/gdrive/My Drive/Colab Notebooks/deepfake/datasets/extract_faces'
FACESDF_DST = '/content/gdrive/My Drive/Colab Notebooks/deepfake/datasets/extract_faces/ffpp_faces.pkl'
CHECKPOINT_DST = '/content/gdrive/My Drive/Colab Notebooks/deepfake/datasets/extract_faces/checkpoint'

### FF++ Preprocessing

In [0]:
def preprocess_ffpp(source_dir, video_dataset_path):
    """
    Preprocessing video dataset
    :param source_dir : Source dir
    :param video_dataset : Path to save the videos DataFrame
    """
    source_dir = Path(source_dir)
    video_dataset_path = Path(video_dataset_path)
    
    # Create ouput folder (if doesn't exist)
    video_dataset_path.parent.mkdir(parents=True, exist_ok=True)

    ## DataFrame
    if video_dataset_path.exists():
        print('Loading video DataFrame')
        df_videos = pd.read_pickle(video_dataset_path)
    else:
        print('Creating video DataFrame')
        df_videos = pd.DataFrame({'path': [f.relative_to(source_dir) for f in source_dir.rglob('*.mp4')]})
        
        df_videos['label'] = df_videos['path'].map(
            lambda x: 1 if x.parts[0] == 'manipulated_sequences' else 0)  # 1 if fake, otherwise 0
        
        source = df_videos['path'].map(lambda x: x.parts[1]).astype('category')
        df_videos['name'] = df_videos['path'].map(lambda x: x.with_suffix('').parts[-1])
        df_videos['path'] = df_videos['path'].map(lambda x: str(x))

        df_videos['original'] = -1 * np.ones(len(df_videos), dtype=np.int16)
        df_videos.loc[(df_videos['label'] == 1) & (source != 'DeepFakeDetection'), 'original'] = \
            df_videos[(df_videos['label'] == 1) & (source != 'DeepFakeDetection')]['name'].map(
                lambda x: df_videos.index[np.flatnonzero(df_videos['name'] == x.split('_')[0])[0]]
            )
        df_videos.loc[(df_videos['label'] == 1) & (source == 'DeepFakeDetection'), 'original'] = \
            df_videos[(df_videos['label'] == 1) & (source == 'DeepFakeDetection')]['name'].map(
                lambda x: df_videos.index[
                    np.flatnonzero(df_videos['name'] == x.split('_')[0] + '__' + x.split('__')[1])[0]]
            )

        print('Saving video DataFrame to {}'.format(video_dataset_path))
        df_videos.to_pickle(str(video_dataset_path))

    print('Real videos: {:d}'.format(sum(df_videos['label'] == 0)))
    print('Fake videos: {:d}'.format(sum(df_videos['label'] == 1)))

### Extract faces

In [0]:
def extract_faces_on_video(video_df, source_dir, faces_dir, checkpoint_dir, blazeface, num_frames, face_size=224, margin=0.25):
    video_idx, video_df = video_df
    faces_checkpoint_path = Path(checkpoint_dir).joinpath(video_df['path'].split('.')[0] + '_faces.pkl')
    if faces_checkpoint_path.exists():
        faces = pd.read_pickle(faces_checkpoint_path)
        return faces
    
    reader = cv2.VideoCapture(str(Path(source_dir).joinpath(video_df['path'])))
    frame_count = int(reader.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_idx = np.unique(np.linspace(0, frame_count - 1, num_frames, dtype=np.int))
    # Get the frames choosen
    frames, idx = [], 0
    while reader.grab():
        if idx in frame_idx:
            ret, frame = reader.retrieve()
            if not ret or frame is None:
                print("Error retrieving frame %d from movie %s" % (frame_idx, path))
                break
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        idx += 1
    frames = np.stack(frames)

    target_w, target_h = blazeface.input_size
    # For each frame, get some tiles with target_w x target_h as size
    num_frames, height, width, _ = frames.shape

    split_size = min(height, width, 720)
    x_step = (width - split_size) // 2
    y_step = (height - split_size) // 2
    num_h = (height - split_size) // y_step + 1 if y_step > 0 else 1
    num_w = (width - split_size) // x_step + 1 if x_step > 0 else 1

    tiles = np.zeros((num_frames * num_h * num_w, target_h, target_w, 3), dtype=np.uint8)
    i = 0
    for f in range(num_frames):
        y = 0
        for _ in range(num_h):
            x = 0
            for __ in range(num_w):
                crop = frames[f, y:y + split_size, x:x + split_size, :]
                tiles[i] = cv2.resize(crop, (target_w, target_h), interpolation=cv2.INTER_AREA)
                x += x_step
                i += 1
            y += y_step

    # Run the face detector. The result is a list of PyTorch tensors
    detections = blazeface.predict_on_batch(tiles, apply_nms=False)
    # Convert the detections from 128x128 back to the original frame size
    for i in range(len(detections)):
        # ymin, xmin, ymax, xmax
        for k in range(2):
            detections[i][:, k * 2] = (detections[i][:, k * 2] * target_h) * split_size / target_h
            detections[i][:, k * 2 + 1] = (detections[i][:, k * 2 + 1] * target_w) * split_size / target_w

    # Because we have several tiles for each frame, combine the predictions from these tiles.
    combined_detections = []
    i = 0
    for f in range(num_frames):
        detections_for_frame = []
        y = 0
        for _ in range(num_h):
            x = 0
            for __ in range(num_w):
                # Adjust the coordinates based on the split positions.
                if detections[i].shape[0] > 0:
                    for k in range(2):
                        detections[i][:, k * 2] += y
                        detections[i][:, k * 2 + 1] += x
                
                detections_for_frame.append(detections[i])
                x += x_step
                i += 1
            y += y_step
            
        combined_detections.append(torch.cat(detections_for_frame))
    if len(combined_detections) == 0:
        return None
    detections = blazeface.nms(combined_detections)
    # Crop the faces out of the original frame.
    faces = []
    for i in range(len(detections)):
        offset = torch.round(margin * (detections[i][:, 2] - detections[i][:, 0])) # margin 0.2
        detections[i][:, 0] = torch.clamp(detections[i][:, 0] - offset * 2, min=0)  # ymin
        detections[i][:, 1] = torch.clamp(detections[i][:, 1] - offset, min=0)  # xmin
        detections[i][:, 2] = torch.clamp(detections[i][:, 2] + offset, max=height)  # ymax
        detections[i][:, 3] = torch.clamp(detections[i][:, 3] + offset, max=width)  # xmax
        
        # Get the first best scored face
        score, face, detection = 0, None, None
        for j in range(len(detections[i])):
            if score < detections[i][j][16].cpu():
                detection = detections[i][j].cpu()
                ymin, xmin, ymax, xmax = detection[:4].cpu().numpy().astype(np.int)
                face = frames[i][ymin:ymax, xmin:xmax, :]
                score = detection[16]
                break
        if face is not None:
            image = Image.fromarray(face)
            # Crop the image to face_size x face_size
            top, left, bottom, right = detection[:4].cpu().numpy().astype(np.int)
            x_ctr = (left + right) // 2
            y_ctr = (top + bottom) // 2
            new_top = max(y_ctr - face_size // 2, 0)
            new_bottom = min(new_top + face_size, height)
            new_left = max(x_ctr - face_size // 2, 0)
            new_right = min(new_left + face_size, width)
            image.crop([new_left, new_top, new_right, new_bottom])
            # Save image
            face_path = Path(faces_dir).joinpath(video_df['path']).joinpath('frame_{}.jpg'.format(frame_idx[i]))
            face_path.parent.mkdir(parents=True, exist_ok=True)
            image.save(face_path)
            
            faces.append({
                'path': str(Path(video_df['path']).joinpath('frame_{}.jpg'.format(frame_idx[i]))),
                'label': video_df['label'],
                'video': video_idx,
                'original': video_df['original'],
                'frame_index': frame_idx[i],
                'score': float(score.numpy()),
                'detection': detection[:4].cpu().numpy().astype(np.int)
            })
        # Save checkpoint
        faces_checkpoint_path.parent.mkdir(parents=True, exist_ok=True)
        pd.DataFrame(faces).to_pickle(str(faces_checkpoint_path))
        
    return faces

In [0]:
def extract_faces(source_dir, videos_df, faces_dir, faces_df, checkpoint_dir, 
                  frames_per_video=15, batch_size=16, face_size=224, thread_num=4):
    
    if Path(faces_df).exists():
        df_faces = pd.read_pickle(faces_df)
        print('We got {} faces'.format(len(df_faces)))
        print('Faces DataFrame Loaded')
        return df_faces
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    print('Loading video DataFrame')
    df_videos = pd.read_pickle(videos_df)
    
    print('Loading Blazeface model')
    blazeface_net = BlazeFace().to(device)
    blazeface_net.load_weights(BLAZEFACE_WEIGHTS)
    blazeface_net.load_anchors(BLAZEFACE_ANCHORS)    
    blazeface_net.min_score_thresh = 0.8
    
    ## Face extraction
    with ThreadPoolExecutor(thread_num) as pool:
        for batch_idx0 in tqdm(np.arange(start=0, stop=len(df_videos), step=batch_size),
                               desc='Extracting faces'):
            list(pool.map(partial(extract_faces_on_video,
                          source_dir=source_dir,
                          faces_dir=faces_dir,
                          checkpoint_dir=checkpoint_dir,
                          blazeface=blazeface_net,
                          num_frames=frames_per_video,
                          face_size=face_size,
                          ),
                          df_videos.iloc[batch_idx0:batch_idx0 + batch_size].iterrows()))
    
    faces_dataset = []
    for _, df in tqdm(df_videos.iterrows(), total=len(df_videos), desc='Collecting faces results'):
        face_checkpoint = Path(checkpoint_dir).joinpath(df['path'].split('.')[0] + '_faces.pkl')
        if face_checkpoint.exists():
            faces_dataset.append(pd.read_pickle(str(face_checkpoint)))
    df_faces = pd.concat(faces_dataset, axis=0)
    df_faces.to_pickle(faces_df)
    print('We got {} faces'.format(len(df_faces)))
    print('Completed!')
    return df_faces

In [0]:
# preprocess ff++ data
preprocess_ffpp(FFPP_SRC, VIDEODF_SRC)

In [0]:
# Run extraction
df_faces = extract_faces(FFPP_SRC, VIDEODF_SRC, FACES_DST, FACESDF_DST,  CHECKPOINT_DST)

## Dataset processing

In [0]:
IMG_SIZE = 224
BATCH_SIZE = 128

### Split dataset on original video

In [0]:
def make_splits(df_faces, train_ratio=0.7, val_ratio=0.15):
    random_original_videos = np.random.permutation(df_faces[(df_faces['label'] == 0)]['video'].unique())
    train_num = int(len(random_original_videos) * train_ratio)
    val_num = int(len(random_original_videos) * val_ratio)
    train_original = random_original_videos[:train_num]
    val_original = random_original_videos[train_num: train_num + val_num]
    test_original = random_original_videos[train_num + val_num:]
    
    df_train = pd.concat([df_faces[df_faces['original'].isin(train_original)], df_faces[df_faces['video'].isin(train_original)]], ignore_index=True)
    df_val = pd.concat([df_faces[df_faces['original'].isin(val_original)], df_faces[df_faces['video'].isin(val_original)]], ignore_index=True)
    df_test = pd.concat([df_faces[df_faces['original'].isin(test_original)], df_faces[df_faces['video'].isin(test_original)]], ignore_index=True)
    
    return df_train, df_val, df_test

In [0]:
df_faces = pd.read_pickle(FACESDF_DST)

In [0]:
df_faces.head()

In [0]:
df_train, df_val, df_test = make_splits(df_faces)

### Transforms

In [0]:
pre_trained_mean, pre_trained_std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=40, scale=(.9, 1.1), shear=0),
    transforms.RandomPerspective(distortion_scale=0.2),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
    transforms.ToTensor(),
    transforms.RandomErasing(scale=(0.02, 0.16), ratio=(0.3, 1.6)),
    transforms.Normalize(mean=pre_trained_mean, std=pre_trained_std)
])

val_tranform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_trained_mean, std=pre_trained_std)
])

### Dataset

In [0]:
class FFPPDataset(Dataset):
    def __init__(self, df_faces, faces_dir=FACES_DST, transform=None):
        super().__init__()
        self.faces_dir = Path(faces_dir)
        self.data, self.targets = df_faces['path'], df_faces['label']
        self.transform = transform
        
    def __getitem__(self, index):
        img_path, target = self.data[index], self.targets[index]
        target = np.array([target,]).astype(np.float32)
        # read the image
        img = Image.open(str(self.faces_dir.joinpath(img_path)))
        
        if self.transform is not None:
            img = self.transform(img)
        return img, target
    
    def __len__(self):
        return len(self.data)

In [0]:
train_dataset = FFPPDataset(df_train, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=24, pin_memory=True)

valid_dataset = FFPPDataset(df_val, transform=val_tranform)
val_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, num_workers=24, pin_memory=True)

test_dataset = FFPPDataset(df_test, transform=val_tranform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=24, pin_memory=True)

## Entraîner le réseau EfficientNet

Modèle pré-entraîner nommé : efficientnet-b4




In [0]:
# Training configuration
# number of epochs to train for
num_epochs = 10

# learning rate
learning_rate = 1e-4

# loss_function
criterion = nn.BCEWithLogitsLoss()

In [0]:
def train(net, model_name):
  model_path = '/content/gdrive/My Drive/Colab Notebooks/deepfake/models/' + model_name + '/best_epoch_{epoch}.ckpt'
  Path(model_path).parent.mkdir(parents=True, exist_ok=True)
  logs_file = '/content/gdrive/My Drive/Colab Notebooks/deepfake/logs/' + model_name + '/log.tsv'
  Path(logs_file).parent.mkdir(parents=True, exist_ok=True)

  callbacks = [
    # Save the weights in a new file when the current model is better than all previous models.
    ModelCheckpoint(model_path, monitor='val_bin_acc', mode='max', save_best_only=True, restore_best=True, verbose=True, temporary_filename=model_name + '_best_epoch.ckpt.tmp'),
    
    # Save the losses and accuracies for each epoch in a TSV.
    CSVLogger(logs_file, separator='\t'),
  ]

  params = (p for p in net.parameters() if p.requires_grad)
  optimizer = optim.Adam(params, lr=learning_rate, weight_decay=0.000001)

  model = Model(net, optimizer, criterion, batch_metrics=['bin_acc'])
  model.to(device)

  if Path(logs_file).exists():
    logs = pd.read_csv(logs_file, sep='\t')
    epochs = list(logs['epoch'])
    if len(epochs) != 0:
      best_epoch_idx = logs['val_bin_acc'].idxmax()
      best_epoch = int(logs.loc[best_epoch_idx]['epoch'])
      model.load_weights(model_path.format(epoch=best_epoch))
      if num_epochs not in epochs:
        # Train
        model.fit_generator(train_loader, val_loader, epochs=num_epochs, initial_epoch=epochs[-1], callbacks=callbacks)
    else:
      model.fit_generator(train_loader, val_loader, epochs=num_epochs, callbacks=callbacks)
  else:
    model.fit_generator(train_loader, val_loader, epochs=num_epochs, callbacks=callbacks)

  logs = pd.read_csv(logs_file, sep='\t')

  best_epoch_idx = logs['val_bin_acc'].idxmax()
  best_epoch = int(logs.loc[best_epoch_idx]['epoch'])
  print("Best epoch: %d" % best_epoch)

  metrics = ['loss', 'val_loss']
  plt.plot(logs['epoch'], logs[metrics])
  plt.legend(metrics)
  plt.title('Loss')
  plt.show()

  metrics = ['bin_acc', 'val_bin_acc']
  plt.plot(logs['epoch'], logs[metrics])
  plt.legend(metrics)
  plt.title('Accuracy')
  plt.show()

  return model

In [0]:
def test(model, test_loader):
  test_loss, test_acc = model.evaluate_generator(test_loader)
  print('Test:\n\tLoss: {}\n\tAccuracy: {}'.format(test_loss, test_acc))

### Freezant tous les paramètres de convolution

In [0]:
efficientnet_a = EfficientNet.from_pretrained('efficientnet-b4')
efficientnet_a._fc = nn.Linear(efficientnet_a._conv_head.out_channels, 1)

for name, param in efficientnet_a.named_parameters():
  if not name.startswith('_fc'):
    param.requires_grad = False

model_a = train(efficientnet_a, 'efficientnet_a')
test(model_a, test_loader)

### En réentraînant quelques blocs

Nous allons réentraîner les 8 derniers blocs avec les deux dernières couches (une couche de convolution et le classificateur)

In [0]:
efficientnet_b = EfficientNet.from_pretrained('efficientnet-b4')
efficientnet_b._fc = nn.Linear(efficientnet_b._conv_head.out_channels, 1)

names = [name for name, _ in list(efficientnet_b.named_parameters())[-109:]]
for name, param in efficientnet_b.named_parameters():
  if name not in names:
    param.requires_grad = False

model_b = train(efficientnet_b, 'efficientnet_b')
test(model_b, test_loader)

Nous allons réentraîner les 16 derniers blocs avec les deux dernières couches (une couche de convolution et le classificateur)

In [0]:
efficientnet_c = EfficientNet.from_pretrained('efficientnet-b4')
efficientnet_c._fc = nn.Linear(efficientnet_c._conv_head.out_channels, 1)

names = [name for name, _ in list(efficientnet_c.named_parameters())[-213:]]
for name, param in efficientnet_c.named_parameters():
  if name not in names:
    param.requires_grad = False

model_c = train(efficientnet_c, 'efficientnet_c')
test(model_c, test_loader)

# Simple mécanisme d'attention

In [0]:
class EfficientNetSimpleAtt(EfficientNet):
    def init_att(self, width: int):
      """
        Initialize attention
        :param depth: attention width
        :return:
      """
        self.att_block_idx = 9
        if width == 0:
            self.attconv = nn.Conv2d(kernel_size=1, in_channels=56, out_channels=1)
        else:
            attconv_layers = []
            for i in range(width):
                attconv_layers.append(
                    ('conv{:d}'.format(i), nn.Conv2d(kernel_size=3, padding=1, in_channels=56, out_channels=56)))
                attconv_layers.append(
                    ('relu{:d}'.format(i), nn.ReLU(inplace=True)))
            attconv_layers.append(('conv_out', nn.Conv2d(kernel_size=1, in_channels=56, out_channels=1)))
            self.attconv = nn.Sequential(OrderedDict(attconv_layers))

    def get_attention(self, inputs):
        att = None
        x = self._swish(self._bn0(self._conv_stem(inputs)))

        # Blocks
        for idx, block in enumerate(self._blocks):
            drop_connect_rate = self._global_params.drop_connect_rate
            if drop_connect_rate:
                drop_connect_rate *= float(idx) / len(self._blocks)
            x = block(x, drop_connect_rate=drop_connect_rate)
            if idx == self.att_block_idx:
                att = torch.sigmoid(self.attconv(x))
                break

        return att

    def extract_features(self, inputs):
        # Stem
        x = self._swish(self._bn0(self._conv_stem(inputs)))

        # Blocks
        for idx, block in enumerate(self._blocks):
            drop_connect_rate = self._global_params.drop_connect_rate
            if drop_connect_rate:
                drop_connect_rate *= float(idx) / len(self._blocks)
            x = block(x, drop_connect_rate=drop_connect_rate)
            if idx == self.att_block_idx:
                att = torch.sigmoid(self.attconv(x))
                x = x * att

        # Head
        x = self._swish(self._bn1(self._conv_head(x)))

        return x

In [0]:
efficientnet_e = EfficientNetSimpleAtt.from_pretrained('efficientnet-b4')
efficientnet_e._fc = nn.Linear(efficientnet_e._conv_head.out_channels, 1)
efficientnet_e.init_att(width)

names = [name for name, _ in list(efficientnet_e.named_parameters())[-221:]]
for name, param in efficientnet_e.named_parameters():
  if name not in names:
    param.requires_grad = False

model_e = train(efficientnet_e, 'efficientnet_e')
test(model_e, test_loader)

## Attention spatiale : Spatial Transfomer Network

In [0]:
class EfficientNetAtt(EfficientNet):
    def __init__(self, blocks_args=None, global_params=None):
        super().__init__(blocks_args, global_params)
        
    def init_spatial_transformer(self):
        self.block_idx = 20
        
        # Spatial transformer localization-network
        self.localization = nn.Sequential(
            nn.Conv2d(160, 160, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(160, 80, kernel_size=3),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )
        
        # Regressor for the 3 * 2 affine matrix
        self.fc_loc = nn.Sequential(
            nn.Linear(80 * 1 * 1, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )
        
        # Initialize the weights/bias with identity transformation
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))
    
    def stn(self, inputs):
        inputs = inputs.clone()
        xs = self.localization(inputs)
        xs = xs.view(-1, 80 * 1 * 1)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, inputs.size(), align_corners=False)
        x = F.grid_sample(inputs, grid, align_corners=False)

        return x
    
    def extract_features(self, inputs):
        """ Returns output of the final convolution layer """

        # Stem
        x = self._swish(self._bn0(self._conv_stem(inputs)))

        # Blocks
        for idx, block in enumerate(self._blocks):
            drop_connect_rate = self._global_params.drop_connect_rate
            if drop_connect_rate:
                drop_connect_rate *= float(idx) / len(self._blocks)
            x = block(x, drop_connect_rate=drop_connect_rate)
            if idx == self.block_idx:
                x *= self.stn(x)

        # Head
        x = self._swish(self._bn1(self._conv_head(x)))

        return x

In [0]:
efficientnet_d = EfficientNetAtt.from_pretrained('efficientnet-b4')
efficientnet_d._fc = nn.Linear(efficientnet_d._conv_head.out_channels, 1)
efficientnet_d.init_spatial_transformer()

names = [name for name, _ in list(efficientnet_d.named_parameters())[-221:]]
for name, param in efficientnet_d.named_parameters():
  if name not in names:
    param.requires_grad = False

model_d = train(efficientnet_d, 'efficientnet_d')
test(model_d, test_loader)